In [ ]:
!pip install --upgrade google-api-python-client

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
import json
from google.colab import files

In [ ]:
YOUTUBE_API_KEY = "your api key here"

In [ ]:
# EXtract function
def get_trending_videos(region_code='IN', max_results=50):
    youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

    request = youtube.videos().list(
        part='snippet,statistics',
        chart='mostPopular',
        regionCode=region_code,
        maxResults=max_results
    )

    response = request.execute()

    return response['items']


In [ ]:
# Pull trending videos for the India
trending_data = get_trending_videos(region_code='IN')

# Check how many videos were pulled
print(f"Fetched {len(trending_data)} videos.")


In [ ]:
#first video's data
import pprint
pprint.pprint(trending_data[0])

In [ ]:
# Transform function
def transform_trending_data(raw_data):
    videos = []

    for video in raw_data:
        snippet = video.get('snippet', {})
        stats = video.get('statistics', {})

        videos.append({
            'video_id': video.get('id'),
            'title': snippet.get('title'),
            'channel_title': snippet.get('channelTitle'),
            'published_at': snippet.get('publishedAt'),
            'category_id': snippet.get('categoryId'),
            'view_count': stats.get('viewCount'),
            'like_count': stats.get('likeCount'),
            'comment_count': stats.get('commentCount'),
        })

    df = pd.DataFrame(videos)

    # Convert numerical columns from strings to integers (if they exist)
    for col in ['view_count', 'like_count', 'comment_count']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

In [ ]:
# Apply transform function
df_trending = transform_trending_data(trending_data)
df_trending.head()

In [ ]:
df_trending.info()
df_trending.isnull().sum()

In [ ]:
import sqlite3

# Connect to SQLite DB 
conn = sqlite3.connect("youtube_trending.db")

# Save DataFrame to a table
df_trending.to_sql("trending_videos", conn, if_exists="replace", index=False)
pd.read_sql_query("SELECT * FROM trending_videos LIMIT 5", conn)

In [ ]:
files.download("youtube_trending.db")

In [ ]:
conn = sqlite3.connect("youtube_trending.db")
df_trending = pd.read_sql_query("SELECT * FROM trending_videos", conn)

In [ ]:
# Visualize top 10 most viewed videos
top_views = df_trending.sort_values(by='view_count', ascending=False).head(10)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.barplot(data=top_views, x='view_count', y='title', palette='viridis')
plt.title('Top 10 Most Viewed Trending Videos in India')
plt.xlabel('Views')
plt.ylabel('Video Title')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize top 10 most liked videos
top_likes = df_trending.sort_values(by='like_count', ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(data=top_likes, x='like_count', y='title', palette='magma')
plt.title('Top 10 Most Liked Trending Videos in India')
plt.xlabel('Likes')
plt.ylabel('Video Title')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize top channels by numbers of trending videos
top_channels = df_trending['channel_title'].value_counts().head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_channels.values, y=top_channels.index, palette='coolwarm')
plt.title('Channels with Most Videos on Trending List')
plt.xlabel('Number of Videos')
plt.ylabel('Channel Name')
plt.tight_layout()
plt.show()